In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os import path
#%matplotlib notebook

from brainpipe.system import study
from brainpipe.visual import BorderPlot, addLines
from brainpipe.feature import TF
from brainpipe.feature import PLV
from brainpipe import *

In [3]:
st = study('Olfacto')
pathdata = path.join(st.path, 'database/TS_E_all_by_odor_th40_art400_30_250_Good_Bad_EpiScore_Expi/')
path2save = path.join(st.path, 'feature/PLV_01_2018/')

test = False
if test == True:
    subjects = ['VACJ']
    freq = [[2, 4]] 
    fname = ['delta']
    
elif test == False :
    subjects = ['SEMC','PIRJ','LEFC','MICP','CHAF','VACJ'] 
    freq = [[2, 4], [5, 7], [8, 13], [13, 30], [30, 60], [60, 150]] 
    fname = ['delta', 'theta', 'alpha', 'beta', 'gamma30-60', 'gamma60-150'] 

n_perm = 1
sf = 512
method = 'hilbert'
conds = ['bad', 'good']

for su in subjects:
    mat_good = np.load(path.join(pathdata, su+'_odor_good_bipo_sel.npz'))
    all_bad = np.load(path.join(pathdata, su+'_odor_bad_bipo_sel.npz'))['x']
    all_good, labels, channels = mat_good['x'], mat_good['label'], mat_good['channel']
    print(all_good.shape, all_bad.shape)
    n_elec, n_pts, n_trials = all_good.shape
    for elec in range(n_elec):
        for i,f in enumerate(freq):
            print('--» processing :', su, 'elec:',elec, '/',n_elec, fname[i], f)
            elec_good = all_good[elec][np.newaxis,...]
            elec_bad = all_bad[elec][np.newaxis,...]
            print(elec_good.shape, elec_bad.shape)
            elec_label, elec_channel = labels[elec], channels[elec]
            defPlv = PLV(sf=sf, npts=n_pts, f=f, method=method, cycle=3)
            plv_good, _ = defPlv.get(all_good,elec_good,n_perm = n_perm)
            plv_bad, _ = defPlv.get(all_bad,elec_bad,n_perm = n_perm)
            plv_good, plv_bad = np.squeeze(plv_good), np.squeeze(plv_bad)
            print('shape after plv good / bad', plv_good.shape, plv_bad.shape)
            plv_diff = plv_good - plv_bad
            print('shape of plv diff', plv_diff.shape)
            np.save(path2save+'plv_npy_files/'+su+'_good_plv_'+fname[i]+'_'+str(elec_channel)+'_'+str(elec_label)+'_elec_('+str(elec)+').npy',plv_good)
            np.save(path2save+'plv_npy_files/'+su+'_bad_plv_'+fname[i]+'_'+str(elec_channel)+'_'+str(elec_label)+'_elec_('+str(elec)+').npy',plv_bad)

            #PLOT PLV values and labels
            plot_title = 'PLV for '+su+' '+fname[i]+' '+str(elec_channel)+' '+str(elec_label)+' elec('+str(elec)+') '
            fig = plt.figure(figsize=(6,8))
            fig.suptitle(plot_title, fontsize=16, fontweight='bold')
            step = 5500/ elec_good.shape[1]
            time = np.arange(0, 5500, step)

            fig, allax = defPlv.plot2D(fig, plv_diff, cmap='jet', xvec=time, xlabel='Time (ms)',                          
                        vmin=-1., vmax=1., ycb=-20,ylabel='Electrodes', figtitle=plot_title, 
                        cblabel='PLV values', subspace={'top':0.9})
            plt.yticks(np.arange(len(labels))+0.5,labels)
            fig.savefig(path2save+'plv_fig/'+su+'_plv_'+fname[i]+'_'+str(elec_channel)+'_'+str(elec_label)+'_elec_('+str(elec)+').png',dpi=300, bbox_inches='tight')
            plt.clf()
            plt.close()
    del elec, all_bad, all_good, labels, channels, elec_good, elec_bad


-> Olfacto loaded
(64, 2816, 23) (64, 2816, 18)
--» processing : SEMC elec: 0 / 64 delta [2, 4]
(1, 2816, 23) (1, 2816, 18)


KeyboardInterrupt: 